In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df=pd.read_csv('ımdb_reviews_train.csv')
train_df.head()

In [ ]:
valid_df=pd.read_csv('ımdb_reviews_valid.csv')
valid_df.head()

In [ ]:
test_df=pd.read_csv('ımdb_reviews_test_unlabeled.csv')
test_df.head()

In [ ]:
df=pd.concat([train_df,valid_df],sort=False)
df.head()

In [ ]:
print("veri seti boyutu:",df.shape)
print("###############################################")
print(df.info())
print("###############################################")
print(df['sentiment'].value_counts())
print("###############################################")
print(df.isnull().sum())

In [ ]:
# 'sentiment' sütunundaki değerleri düzelt
df['sentiment'] = df['sentiment'].replace({
    '1;': 'positive',
    '0;': 'negative',
    'positive"': 'positive',
    'negative;;': 'negative',
    'positive;;': 'positive',
    '0"': 'negative',
})


# Sonuçları kontrol et
print(df['sentiment'].value_counts())


In [ ]:
import re

def clean_text(text):
    # 1.HTML etiketlerini kaldırma
    text = re.sub(r'<.*?>', '', text)
    
    # 2.URL'leri kaldırma
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 3.Gereksiz boşlukları kaldırma
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 4. Küçük harflere dönüştür
    text = text.lower()
    
    # 5. Noktalama işaretlerini kaldır
    text = re.sub(r'[^\w\s]', '', text)

    return text

In [ ]:
df['review'] = df['review'].apply(clean_text)
print(df['review'])

In [ ]:
"""import nltk
# NLTK verilerini belirli bir dizine indir
nltk.data.path.append('/GANsAugmentation-SentimentAnalysis/nltk_data')  

# Gerekli NLTK bileşenlerini indir
nltk.download('all', download_dir='/GANsAugmentation-SentimentAnalysis/nltk_data')"""

In [ ]:
"""from nltk.tokenize import word_tokenize
from collections import Counter

def remove_repeated_phrases(text):
    # Metni kelimelere böl
    words = word_tokenize(text)
    
    # Her kelimenin kaç kez tekrar ettiğini say
    word_freq = Counter(words)
    print(word_freq)
    
    # Sadece bir kez geçen kelimeleri al
    non_repeated_words = [word for word in words if word_freq[word] == 1]
    
    # Bu kelimeleri birleştirip metni oluştur
    cleaned_text = ' '.join(non_repeated_words)
    
    return cleaned_text

# Örnek kullanım
sample_text = "This is a test. This is only a test."
cleaned_text = remove_repeated_phrases(sample_text)
print(cleaned_text)"""


In [ ]:
df['review'] = df['review'].apply(remove_repeated_phrases)
print(df['review'])

In [ ]:
"""
from textblob import TextBlob
import multiprocessing as mp

def correct_spelling(text):
    text_blob = TextBlob(text)
    return str(text_blob.correct())

def process_in_parallel(data, num_workers):
    with mp.Pool(num_workers) as pool:
        results = pool.map(correct_spelling, data)
    return results

# Örnek kullanım
data = ["Your text data here" ]
num_workers = mp.cpu_count()  # İşlemcinizin çekirdek sayısına göre belirleyin
corrected_data = process_in_parallel(data, num_workers)
print(corrected_data)
"""


In [ ]:
"""from symspellpy import SymSpell, Verbosity

# SymSpell örneği oluştur
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Sözlük yükleme (indirdiğiniz dosyanın tam yolunu belirtin)
sym_spell.load_dictionary("frequency_dictionary_en_82_765.txt", 0, 1)

def correct_spelling_symspell(text):
    corrected_text = []
    for word in text.split():
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_text.append(suggestions[0].term)
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)"""



In [ ]:
df['review'] = df['review'].apply(correct_spelling_symspell)
print(df['review'])

In [ ]:
"""from gensim.models import KeyedVectors
#Synonym Replacement (Sinonim Değiştirme)
#Bir kelimenin anlamını koruyarak metindeki kelimeleri değiştirme

# Google'ın pre-trained Word2Vec modelini indirip yükleyin
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)

def get_similar_words(word):
    if word in model:
        return [similar_word for similar_word, _ in model.most_similar(word)]
    return [word]

def replace_with_similar_words(text):
    words = text.split()
    new_words = []
    for word in words:
        similar_words = get_similar_words(word)
        if similar_words:
            new_words.append(similar_words[0])  # Replace with first similar word
        else:
            new_words.append(word)
    return ' '.join(new_words)

# Test et
#sentence = "This movie was horrible. If it had never been made the world would be a better place. Come on, a flying wagon? What were they thinking? This was a sub-par movie with a horrible hook, and I would like a written apology from the studio that produced this, along with some cookies to help repay me for the time I wasted on this crap fest that I can never get back. If you payed to see this movie, I am truly sorry because I watched it on TV on a Sunday afternoon when I had nothing better to do and it pretty much ruined my whole week. A flying freaking WAGON?!?! And that's supposed to make up for having a horrible mother who cares more about her own screwed up needs than her children? No wonder they don't have enough sense to tell someone he is beating them, their mother teaches them nothing but that what she wants comes before everything else. Absolutely horrible."
#sentence1=clean_text(sentence)
#sentences2=remove_repeated_phrases(sentence1)
#sentences3=correct_spelling_symspell(sentences2)
#print(replace_with_similar_words(sentences3))"""


In [ ]:
"""def get_similar_words(word, threshold=0.6):
    if word in model:
        similar_words = model.most_similar(word)
        # Sadece eşiğin üzerinde olan kelimeleri al
        return [similar_word for similar_word, similarity in similar_words if similarity >= threshold]
    return [word]

def replace_with_similar_words(text):
    words = text.split()
    new_words = []
    for word in words:
        similar_words = get_similar_words(word)
        if similar_words:
            new_words.append(similar_words[0])  # Replace with first similar word
        else:
            new_words.append(word)
    return ' '.join(new_words)

# Test et
sentence = "This movie was horrible. If it had never been made the world would be a better place. Come on, a flying wagon? What were they thinking? This was a sub-par movie with a horrible hook, and I would like a written apology from the studio that produced this, along with some cookies to help repay me for the time I wasted on this crap fest that I can never get back. If you payed to see this movie, I am truly sorry because I watched it on TV on a Sunday afternoon when I had nothing better to do and it pretty much ruined my whole week. A flying freaking WAGON?!?! And that's supposed to make up for having a horrible mother who cares more about her own screwed up needs than her children? No wonder they don't have enough sense to tell someone he is beating them, their mother teaches them nothing but that what she wants comes before everything else. Absolutely horrible."
sentence1 = clean_text(sentence)
sentences2 = remove_repeated_phrases(sentence1)
sentences3 = correct_spelling_symspell(sentences2)
print(replace_with_similar_words(sentences3))"""


In [ ]:
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
import torch
print(torch.__version__)


In [ ]:
"""from transformers import pipeline

# T5 modelini yükle
paraphrase_model = pipeline("text2text-generation", model="t5-base")

# Parafraz işlemi yap
def paraphrase_text(text):
    result = paraphrase_model(f"paraphrase: {text}", max_length=256, num_beams=5, early_stopping=True)
    return result[0]['generated_text']

# Örnek cümle
sentence = "This movie was horrible. If it had never been made the world would be a better place."
paraphrased_sentence = paraphrase_text(sentence)
print(paraphrased_sentence)"""


In [ ]:
"""import random
#noise Insertion (gürültü Ekleme)
#Metne rastgele kelimeler ekleyerek metni bozma

def add_noise(text):
    words = text.split()
    num_words_to_add = int(len(words) * 0.1)
    for _ in range(num_words_to_add):
        random_index = random.randint(0, len(words)-1)
        words.insert(random_index, words[random_index])
    return ' '.join(words)

augmented_review = 'The movie was good and I liked it'
# Örnek kullanımı
noisy_review = add_noise(augmented_review)
print(noisy_review)"""


In [ ]:
import random

def add_noise(text, noise_words):
    words = text.split()
    num_words_to_add = int(len(words) * 0.1)  # Metnin %10'u kadar gürültü kelimesi ekle
    for _ in range(num_words_to_add):
        random_index = random.randint(0, len(words) - 1)
        noise_word = random.choice(noise_words)
        words.insert(random_index, noise_word)
    return ' '.join(words)

# Gürültü için kullanılacak kelimeler listesi
noise_words = [
    'um', 'uh', 'like', 'so', 'you know', 'actually', 'basically',
    'seriously', 'literally', 'well', 'honestly', 'truly', 'really',
    'I mean', 'basically', 'just', 'kinda', 'sorta', 'probably',
    'maybe', 'definitely', 'literally', 'figuratively', 'almost'
]

# Örnek metin
#augmented_review = 'her acting was good and I liked it'
#noisy_review = add_noise(augmented_review, noise_words)
#print(noisy_review)


In [ ]:
df['review'] = df['review'].apply(add_noise, noise_words=noise_words)
print(df['review'])

In [ ]:
"""import torch
import torch.nn as nn
import torch.optim as optim

# Generator Ağı
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Discriminator Ağı
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Model Boyutları
input_size = 100
hidden_size = 50
output_size = 100

# Modelleri Tanımla
generator = Generator(input_size, hidden_size, output_size)
discriminator = Discriminator(output_size, hidden_size)

# Kayıp Fonksiyonları ve Optimizatörler
criterion = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=0.001)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

# Eğitim döngüsü (basit örnek)
for epoch in range(10000):
    # Gerçek ve sahte cümleler için etiketler
    real_labels = torch.ones(1, 1)
    fake_labels = torch.zeros(1, 1)

    # Gerçek verilerden (cümlelerden) örnek al
    real_data = torch.randn(1, output_size)  # Bu, gerçek veri yerine rastgele bir placeholder
    fake_data = generator(torch.randn(1, input_size))

    # Discriminator'u eğit
    d_optimizer.zero_grad()
    real_output = discriminator(real_data)
    fake_output = discriminator(fake_data.detach())
    d_loss_real = criterion(real_output, real_labels)
    d_loss_fake = criterion(fake_output, fake_labels)
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    d_optimizer.step()

    # Generator'u eğit
    g_optimizer.zero_grad()
    fake_output = discriminator(fake_data)
    g_loss = criterion(fake_output, real_labels)
    g_loss.backward()
    g_optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

# Eğitim tamamlandığında cümleleri paraphrase et
new_sentence = generator(torch.randn(1, input_size))
print("Generated Sentence Paraphrase:", new_sentence)"""


In [ ]:
#df['review'] = df['review'].apply(lambda x: generator(torch.randn(1, input_size)))

In [ ]:
df['review'].head()